In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

In [4]:
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

In [5]:
import numpy as np
import pandas as pd

In [36]:
butte = pd.read_csv("/content/drive/MyDrive/Wildfire (1)/Datasets/merged_tp_precip_wind_fmc_Butte.csv")
calaveras = pd.read_csv("/content/drive/MyDrive/Wildfire (1)/Datasets/merged_tp_precip_wind_fmc_Calaveras.csv")
dorado = pd.read_csv("/content/drive/MyDrive/Wildfire (1)/Datasets/merged_tp_precip_wind_fmc_Dorado.csv")
fresno = pd.read_csv("/content/drive/MyDrive/Wildfire (1)/Datasets/merged_tp_precip_wind_fmc_Fresno.csv")
humboldt = pd.read_csv("/content/drive/MyDrive/Wildfire (1)/Datasets/merged_tp_precip_wind_fmc_Humboldt.csv")
kern = pd.read_csv("/content/drive/MyDrive/Wildfire (1)/Datasets/merged_tp_precip_wind_fmc_KERN.csv")
lassen = pd.read_csv("/content/drive/MyDrive/Wildfire (1)/Datasets/merged_tp_precip_wind_fmc_Lassen.csv")
modoc = pd.read_csv("/content/drive/MyDrive/Wildfire (1)/Datasets/merged_tp_precip_wind_fmc_Modoc.csv")
nevada = pd.read_csv("/content/drive/MyDrive/Wildfire (1)/Datasets/merged_tp_precip_wind_fmc_Nevada.csv")
plumas = pd.read_csv("/content/drive/MyDrive/Wildfire (1)/Datasets/merged_tp_precip_wind_fmc_Plumas.csv")
riverside = pd.read_csv("/content/drive/MyDrive/Wildfire (1)/Datasets/merged_tp_precip_wind_fmc_Riverside.csv")
bernardino = pd.read_csv("/content/drive/MyDrive/Wildfire (1)/Datasets/merged_tp_precip_wind_fmc_San Bernardino.csv")
diego = pd.read_csv("/content/drive/MyDrive/Wildfire (1)/Datasets/merged_tp_precip_wind_fmc_San Diego.csv")
shasta = pd.read_csv("/content/drive/MyDrive/Wildfire (1)/Datasets/merged_tp_precip_wind_fmc_Shasta.csv")
siskiyou = pd.read_csv("/content/drive/MyDrive/Wildfire (1)/Datasets/merged_tp_precip_wind_fmc_Siskiyou.csv")
chelan = pd.read_csv("/content/drive/MyDrive/Wildfire (1)/Datasets/merged_tp_precip_wind_fmc_Chelan.csv")
clearwater = pd.read_csv("/content/drive/MyDrive/Wildfire (1)/Datasets/merged_tp_precip_wind_fmc_Clearwater.csv")
coconino = pd.read_csv("/content/drive/MyDrive/Wildfire (1)/Datasets/merged_tp_precip_wind_fmc_Coconino.csv")
idaho = pd.read_csv("/content/drive/MyDrive/Wildfire (1)/Datasets/merged_tp_precip_wind_fmc_Idaho.csv")
mohave = pd.read_csv("/content/drive/MyDrive/Wildfire (1)/Datasets/merged_tp_precip_wind_fmc_Mohave.csv")
trinity = pd.read_csv("/content/drive/MyDrive/Wildfire (1)/Datasets/merged_tp_precip_wind_fmc_Trinity.csv")
valley = pd.read_csv("/content/drive/MyDrive/Wildfire (1)/Datasets/merged_tp_precip_wind_fmc_Valley.csv")
yukon = pd.read_csv("/content/drive/MyDrive/Wildfire (1)/Datasets/merged_tp_precip_wind_fmc_yukon-koyukuk.csv")
all = pd.read_csv("/content/drive/MyDrive/Wildfire (1)/Datasets/merged_tp_precip_wind_fmc_all.csv")

In [42]:
all = pd.read_csv("/content/drive/MyDrive/Wildfire (1)/Datasets/merged_tp_precip_wind_fmc_all.csv")

In [43]:

def append_dataframes(dataframes):
    if len(dataframes) == 0:
        return None
    elif len(dataframes) == 1:
        return dataframes[0]
    else:
        combined_df = pd.concat(dataframes, ignore_index=True)
        return combined_df


dataframes_list = [butte, calaveras, dorado, fresno, humboldt, kern, lassen, modoc, dorado, nevada, plumas, riverside, bernardino, diego, shasta, siskiyou, chelan, clearwater, coconino, idaho, mohave, trinity, valley, yukon, all]

fire_data = append_dataframes(dataframes_list)

In [ ]:
butte.head(5)

In [ ]:
calaveras.head(5)

In [ ]:
fresno.head(5)

In [ ]:
humboldt.head(5)

In [ ]:
kern.head(5)

In [ ]:
lassen.head(5)

In [ ]:
modoc.head(5)

In [ ]:
dorado.head(5)

In [ ]:
nevada.head(5)

In [ ]:
plumas.head(5)

In [ ]:
riverside.head(5)

In [ ]:
bernardino.head(5)

In [ ]:
diego.head(5)

In [ ]:
shasta.head(5)

In [ ]:
siskiyou.head(5)

In [ ]:
chelan.head(5)

In [ ]:
clearwater.head(5)

In [ ]:
coconino.head(5)

In [ ]:
idaho.head(5)

In [ ]:
mohave.head(5)

In [ ]:
trinity.head(5)

In [ ]:
valley.head(5)

In [ ]:
yukon.head(5)

In [ ]:
fire_data.info()

In [47]:
fire_data.head(500)

,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,FIPS,DATE,tmax,tmin,tavg,prcp,aws,fmc
0,2.0,B,39.735278,-121.336389,6007.0,20080621,389.0,222.0,306.0,0.0,1.245353,96.0
1,1.0,B,39.533339,-121.530711,6007.0,20010808,411.0,228.0,322.0,0.0,2.611998,0.0
2,12.0,C,39.355089,-121.772239,6007.0,20041012,328.0,133.0,233.0,0.0,3.008472,0.0
3,0.1,A,39.790389,-121.515500,6007.0,20041011,322.0,133.0,228.0,0.0,5.034409,0.0
4,787.5,E,39.777800,-121.552470,6007.0,20080621,389.0,222.0,306.0,0.0,1.245353,96.0
...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.1,A,39.921469,-121.728597,6007.0,20070228,389.0,222.0,306.0,0.0,3.104664,110.0
496,1.0,B,39.572953,-121.313275,6007.0,20070401,389.0,222.0,306.0,0.0,1.793009,102.0
497,0.5,B,39.980181,-121.550731,6007.0,20040625,333.0,161.0,250.0,0.0,4.080243,0.0
498,0.1,A,39.849939,-121.588689,6007.0,19980726,383.0,211.0,300.0,0.0,1.055472,0.0


In [17]:


fire_data.drop(["START_DATE"], axis = 1, inplace = True)
fire_data.drop(["END_DATE"], axis = 1, inplace = True)

In [45]:
fire_data.drop(["Unnamed: 0.3"], axis = 1, inplace = True)
fire_data.drop(["Unnamed: 0.2"], axis = 1, inplace = True)
fire_data.drop(["Unnamed: 0.1"], axis = 1, inplace = True)
fire_data.drop(["Unnamed: 0"], axis = 1, inplace = True)
fire_data.drop(["STATE"], axis = 1, inplace = True)
fire_data.drop(["START_DATE"], axis = 1, inplace = True)
fire_data.drop(["END_DATE"], axis = 1, inplace = True)



In [46]:
fire_data.drop(["Unnamed: 0.4"], axis = 1, inplace = True)
fire_data.drop(["Unnamed: 0.5"], axis = 1, inplace = True)
fire_data.drop(["Unnamed: 0.6"], axis = 1, inplace = True)
fire_data.drop(["Unnamed: 0.7"], axis = 1, inplace = True)
fire_data.drop(["Unnamed: 0.8"], axis = 1, inplace = True)
fire_data.drop(["Unnamed: 0.9"], axis = 1, inplace = True)

In [ ]:
def classify_ranges(df, column_name, ranges, classification_column):
    for range_name, range_values in ranges.items():
        df.loc[df[column_name].between(*range_values), classification_column] = range_name

# Example usage


classification_ranges = {
    'A': (0, 0.25),
    'B': (0.2600000001, 9),
    'C': (9.000000001, 99),
    'D': (99.000000001, 299),
    'E': (299.000000001, 999),
    'F': (999.000000001, 4999),
    'G': (4999.000000001, 10000),
    'G': (10000.000000001, 1000000000)

}
  # Creating a new column to store the classifications
classify_ranges(fire_data, 'FIRE_SIZE', classification_ranges, 'FIRE_SIZE_CLASS')


In [48]:
def classify_ranges(df, column_name, ranges, classification_column):
    for range_name, range_values in ranges.items():
        df.loc[df[column_name].between(*range_values), classification_column] = range_name

# Example usage


classification_ranges = {
    'A': (0, 99),
    'B': (99.000000000001, 999),
    'C': (999.000000000001, 99999999999999)



}
  # Creating a new column to store the classifications
classify_ranges(fire_data, 'FIRE_SIZE', classification_ranges, 'FIRE_SIZE_CLASS')


In [ ]:
def classify_ranges(df, column_name, ranges, classification_column):
    for range_name, range_values in ranges.items():
        df.loc[df[column_name].between(*range_values), classification_column] = range_name

# Example usage


classification_ranges = {
    'A': (0, .25),
    'B': (.25000000001, 999999999)




}
  # Creating a new column to store the classifications
classify_ranges(fire_data, 'FIRE_SIZE', classification_ranges, 'FIRE_SIZE_CLASS')


In [50]:
fire_data.drop(["FIRE_SIZE"], axis = 1, inplace = True)

In [49]:
fire_data.groupby("FIRE_SIZE_CLASS").count()

,FIRE_SIZE,LATITUDE,LONGITUDE,FIPS,DATE,tmax,tmin,tavg,prcp,aws,fmc
FIRE_SIZE_CLASS,,,,,,,,,,,
A,129989,129989,129989,129989,129989,129975,129975,129975,129975,129989,129989
B,8958,8958,8958,8958,8958,8958,8958,8958,8958,8958,8958
C,1785,1785,1785,1785,1785,1785,1785,1785,1785,1785,1785


In [51]:
fire_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140732 entries, 0 to 140731
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   FIRE_SIZE_CLASS  140732 non-null  object 
 1   LATITUDE         140732 non-null  float64
 2   LONGITUDE        140732 non-null  float64
 3   FIPS             140732 non-null  float64
 4   DATE             140732 non-null  object 
 5   tmax             140718 non-null  float64
 6   tmin             140718 non-null  float64
 7   tavg             140718 non-null  float64
 8   prcp             140718 non-null  float64
 9   aws              140732 non-null  float64
 10  fmc              140732 non-null  float64
dtypes: float64(9), object(2)
memory usage: 11.8+ MB


In [52]:
fire_data.dropna(inplace = True)
fire_data['FIRE_SIZE_CLASS'] = fire_data['FIRE_SIZE_CLASS'].replace({'A':0 , 'B':1 , 'C':2 ,'D':2 , 'E':2 , 'F':2 , 'G':2 })

In [53]:
fire_data["DATE"] = fire_data["DATE"].astype(float)

In [54]:
x = fire_data.drop("FIRE_SIZE_CLASS", axis=1)
y = fire_data["FIRE_SIZE_CLASS"]

In [55]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size = .3, random_state = 101)

In [56]:
X_train = torch.tensor(X_train.values, dtype = torch.float)
Y_train = torch.tensor(Y_train.values, dtype = torch.int64)
X_test = torch.tensor(X_test.values, dtype = torch.float)
Y_test = torch.tensor(Y_test.values, dtype = torch.int64)

In [ ]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.hidden1 = nn.Linear(X_train.shape[1], 16)
    self.hidden2 = nn.Linear(16, 1)


  def forward(self, x):
    x = torch.relu(self.hidden1(x))
    x = self.hidden2(x)
    return x

In [ ]:
model = NeuralNetwork()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), .0005)
print(model)

In [ ]:
#Y_train = torch.flatten(Y_train)
#Y_test = torch.flatten(Y_test)

In [ ]:
train_data = TensorDataset(X_train, Y_train)
train_loader = DataLoader(train_data, batch_size = 32)

In [ ]:
epochs = 20
loss_list = np.zeros((epochs,))

for epoch in range(epochs):
  running_loss = 0
  model.train(True)
  for inputs, targets in train_loader:
    optimizer.zero_grad() # reset optimizer gradients
    outputs = model(inputs) # forward pass data through nodes and layers
    outputs = torch.flatten(outputs)
    loss = loss_fn(outputs, targets) # compute loss
    loss_list[epoch] = loss.item()
    running_loss += loss.item()
    loss.backward() #backpropagation
    optimizer.step() # update weights

  if epoch%10 == 0:
    print(f"Epoch [{epoch}/10000], Loss: {running_loss/len(train_loader)}")

In [57]:

class Softmax(torch.nn.Module):
    "custom softmax module"
    def __init__(self, n_inputs, n_outputs):
        super().__init__()
        self.linear = torch.nn.Linear(n_inputs, n_outputs)

    def forward(self, x):
        pred = self.linear(x)
        return pred

In [58]:
model_softmax = Softmax(10, 3)
model_softmax.state_dict()

OrderedDict([('linear.weight',
              tensor([[-0.0169, -0.0623,  0.1991, -0.1744,  0.1987,  0.2097,  0.0720,  0.0764,
                        0.0036, -0.1522],
                      [ 0.0923,  0.0732, -0.1442, -0.2251,  0.2493,  0.2887, -0.2450,  0.1831,
                        0.0272, -0.0725],
                      [-0.0202,  0.2267,  0.0557,  0.1877, -0.2963, -0.2142, -0.1313, -0.2443,
                        0.2440, -0.2225]])),
             ('linear.bias', tensor([-0.3022, -0.2140,  0.0847]))])

In [59]:
train_data = TensorDataset(X_train, Y_train)
train_loader = DataLoader(train_data, batch_size = 32)

In [60]:

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_softmax.parameters(), .0005)

In [61]:
Loss = []
epochs = 100
for epoch in range(epochs):
    for x, y in train_loader:
        optimizer.zero_grad()
        y_pred = model_softmax(x)

        loss = loss_fn(y_pred, y)
        Loss.append(loss)
        loss.backward()
        optimizer.step()
    if epoch%10 == 0:
      print("i")

i
i
i
i
i
i
i
i
i
i


In [ ]:
y_pred

In [ ]:
y

In [62]:
pred_model =  model_softmax(X_test)
_, y_pred = pred_model.max(1)

In [ ]:
y_pred

In [64]:
correct = (Y_test == y_pred).sum().item()
acc = correct / Y_test.size(0)
print("model accuracy: ", acc)

model accuracy:  0.9226122797043775


In [65]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import f1_score
model_softmax.eval()

with torch.no_grad():

    actuals = Y_test.tolist()
predictions = model_softmax(X_test)
predictions = torch.tensor(predictions)
actuals = torch.tensor(actuals)

mse = mean_squared_error(actuals, y_pred)
r2 = r2_score(actuals, y_pred)
rmse = np.sqrt(mse)

print('Test MSE:', mse)
print("Test r2:", r2)
print('Test RMSE:', rmse)


Test MSE: 0.1159039226833428
Test r2: -0.07554359843458291
Test RMSE: 0.34044665174347477


<ipython-input-65-2e2099014a7b>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  predictions = torch.tensor(predictions)


In [68]:
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


In [69]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
fire_data.groupby("FIRE_SIZE_CLASS").count()

In [ ]:
fire_data.head(100)

In [ ]:
fire_data.info()

In [72]:
x = fire_data.drop("FIRE_SIZE_CLASS", axis=1)
y = fire_data["FIRE_SIZE_CLASS"]

In [ ]:

# Sample DataFrame

df = fire_data

# Value to check and remove
value_to_remove = 1
column_to_check = 'FIRE_SIZE_CLASS'

# Check for value in the specified column and remove the corresponding row
if value_to_remove in df[column_to_check].values:
    fire_data1 = df[df[column_to_check] != value_to_remove]
    print(f"Removed rows with {value_to_remove} in {column_to_check} column:\n{df}")
else:
    print(f"{value_to_remove} not found in {column_to_check} column")

# Print the updated DataFrame
print("Updated DataFrame:")
print(df)

In [ ]:
fire_data1.head()

In [ ]:
fire_data1.groupby("FIRE_SIZE_CLASS").count()


In [71]:
x = fire_data1.drop("FIRE_SIZE_CLASS", axis=1)
y = fire_data1["FIRE_SIZE_CLASS"]

NameError: ignored

In [73]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size = 0.20, random_state = 101)

In [74]:
lm = LogisticRegression()
lm.fit(X_train, Y_train)

LogisticRegression()

In [75]:
predictions = lm.predict(X_test)

In [76]:
mean_squared_error(Y_test, predictions)

0.11579732802728823

In [77]:
print(classification_report(Y_test, predictions))

              precision    recall  f1-score   support

           0       0.92      1.00      0.96     25983
           1       0.00      0.00      0.00      1795
           2       0.00      0.00      0.00       366

    accuracy                           0.92     28144
   macro avg       0.31      0.33      0.32     28144
weighted avg       0.85      0.92      0.89     28144



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(confusion_matrix(Y_test, predictions))

In [ ]:
print(accuracy_score(Y_test, predictions))

In [ ]:
from sklearn import metrics

In [ ]:
mean_squared_error(Y_test, predictions)

In [ ]:
r2_score(Y_test, predictions)

In [78]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(X_train, Y_train)

DecisionTreeClassifier()

In [79]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [80]:
y_pred_dtc = dtc.predict(X_test)

In [ ]:
X_test

In [81]:
print(classification_report(Y_test, y_pred_dtc))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     25983
           1       0.83      0.85      0.84      1795
           2       0.83      0.80      0.81       366

    accuracy                           0.98     28144
   macro avg       0.88      0.88      0.88     28144
weighted avg       0.98      0.98      0.98     28144



In [82]:
print(confusion_matrix(Y_test, y_pred_dtc))


[[25639   290    54]
 [  268  1519     8]
 [   50    22   294]]


In [83]:
print(accuracy_score(Y_test, y_pred_dtc))


0.9754121660034111


In [84]:
r2_score(Y_test, y_pred_dtc)

0.6688769174053768

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()

In [ ]:
scaler.fit(fire_data.drop('FIRE_SIZE_CLASS',axis=1))

In [ ]:
scaled_features = scaler.transform(fire_data.drop('FIRE_SIZE_CLASS',axis=1))

In [ ]:
fire_data.head()

In [ ]:
df_feat = pd.DataFrame(scaled_features,columns=fire_data.columns[1:])
df_feat.head()

In [ ]:
X = df_feat
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)